In [1]:
import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.embeddings import HuggingFaceEmbeddings
from os import getcwd, environ, listdir
from pathlib import Path
from sys import path

path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager

In [2]:
from yaml import safe_load

def load_config(file_path="./config.yml"):
    with open(file_path, 'r') as file:
        config = safe_load(file)
        for key, value in config.items():
            environ[key] = value
load_config()

In [3]:
TST_DIR = Path(getcwd())
PROJ_DIR = TST_DIR.parent
DB_DIR = TST_DIR / "chroma_test"
DATA_DIR = PROJ_DIR / "data"

## Load docs

In [4]:
embedding_function = HuggingFaceEmbeddings()

c:\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
docs = []
for file in listdir(str(DATA_DIR)):
    loader = TextLoader(str(DATA_DIR / file))
    docs.extend(loader.load())


1225


In [27]:
vectordb = Chroma('parent_docs',
                embedding_function=embedding_function, 
                persist_directory=str(DB_DIR),
                client_settings=Settings(allow_reset=True))
vectordb.reset_collection()
print("There are", vectordb._collection.count(), "in the collection")

There are 0 in the collection


c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [28]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap=200)
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectordb, 
    docstore=store,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter)

In [29]:
retriever.add_documents(docs)

In [22]:
query = "Who are the shards on Roshar?"

In [23]:
response = retriever.invoke(query)

In [24]:

response
# print(response[1].page_content)

[Document(metadata={'source': 'c:\\Users\\Izogie\\Desktop\\Folders\\Projects\\Python\\KB Chat\\data\\Realmatic Theory.txt'}, page_content='Shards \n\nWhile the Shards of Adonalsium exist primarily in the Spiritual Realm, their immense power has notable effects on all three of the Realms. Perhaps most significantly, Realmatic interactions between a Shard and a planet are what define the structure of any Invested Arts associated with that Shard on that planet. Shards can consciously make some changes to this structure, but the extent to which they can interfere is limited. If a Shard used their power to create the planet in the first place (as Preservation and Ruin did with Scadrial), they will be able to exercise more control.'),
 Document(metadata={'source': 'c:\\Users\\Izogie\\Desktop\\Folders\\Projects\\Python\\KB Chat\\data\\Realmatic Theory.txt'}, page_content='While most perpendicularities are created by Shards, they can also be created by other methods. Some Invested Arts, like t

In [21]:
# client = chromadb.PersistentClient(path=str(DB_DIR), settings=Settings(allow_reset=True))
# langchain_chroma = Chroma(
#     client=client,
#     collection_name="Full_docs",
#     embedding_function=
# )

In [ ]:
import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma
from os import getcwd, environ, listdir
from pathlib import Path
from langchain.retrievers import ParentDocumentRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from sys import path
path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager


PROJ_DIR = Path(getcwd()).parent
DB_DIR = PROJ_DIR / "chroma"

from langchain_huggingface import HuggingFaceEmbeddings
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

query = "what does steel do"

docs = []
for filename in listdir(str(PROJ_DIR / "data")):
    loader = TextLoader(str(PROJ_DIR / f"data/{filename}"))
    docs.extend(loader.load())

vectorstore = Chroma.from_documents(
    docs, 
    embedding_function,
    collection_name="full_documents", 
    persist_directory=str(DB_DIR))
print("There are", vectorstore._collection.count(), "in the collection")

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, 
    docstore=store,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter)
retrieved_docs = retriever.invoke("steel")
print(len(list(store.yield_keys())))
print(retrieved_docs)